In [ ]:
import json
import sys
import glob
import nltk
import operator
import math
from random import randint
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')

In [ ]:
text = "american american american american american american american american american american american american american american american american american american american american american american american american american candidate candidate candidate candid"
list_of_idf_files = "dbpedia_res/idf_light.tsv", "dbpedia_res/idf_dark.tsv", "dbpedia_res/idf_non.tsv"
list_of_json_files = ["dbpedia_res/light.json", "dbpedia_res/dark.json", "dbpedia_res/non.json"]

In [ ]:
def create_concept_dictionaries(list_of_json_files):
    list_of_dict = []
    for json_file in list_of_json_files:
        jsonfile = open(json_file)
        json_dict = json.load(jsonfile)
        list_of_dict.append(json_dict)
    return list_of_dict

In [ ]:
def concept_indexer(source):
    target_index = {}
    for dict_per_category in source:
        for instance in dict_per_category['instance']:
            concept_dict = {}
            instance_type = instance['instance']
            name = instance['uri'].split("/")[-1].lower().replace("_"," ").replace("+"," ")
            labels = []
            for label_caps in instance['labels']:
                labels.append(label_caps.lower())
            types = []
            for type_ in instance['types']:
                types.append(type_.split("/")[-1])
            concept_dict.update({"types":types})
            concept_dict.update({"labels":labels})
            target_index[name] = concept_dict
    return target_index

In [ ]:
def term_idf_indexer(list_of_idf_files):
    for idf_file in list_of_idf_files:
        idf_score_data = open(idf_file)
        term_idf_index = {}
        for line in idf_score_data.readlines():
            term_dict = {} 
            term,idf_score = line.strip("\n").split("\t")
            #term_dict["term"] = term
            term_dict["idfscore"] = idf_score            
            term_dict["category"] = idf_file
            term_idf_index[term] = term_dict
        return term_idf_index

In [ ]:
def input_indexer(input_text):
    input_index = {}
    for term in text.split(" "):
        if term not in stopwords.words('english'):
            input_index[term] = text.split(" ").count(term)
    return input_index 

In [ ]:
def match_terms(input_index, term_idf_index):
    matched_terms_dict = {}
    for term_input, tf in input_index.items():
        for term_index, attrib in term_idf_index.items(): 
            if term_input == term_index:
                matched_term = {}
                matched_terms_dict[term_input] = matched_term
                tf_calc, idf_calc = float(tf), float(attrib['idfscore'])
                tf_idf = tf_calc * idf_calc
                matched_term["tf-idf"] = tf_idf
    return matched_terms_dict    

In [ ]:
def xpath_get(mydict, path):
    elem = mydict
    try:
        for x in path.strip("/").split("/"):
            elem = elem.get(x)
    except:
        pass
    return elem

In [58]:
def define_respons_concept(matched_terms_dic):
    interesting_concept = max(matched_terms_dic, key=lambda i: [1])
    types = []
    typesdict = {}
    for concept,attributes in concept_index.items():
        for label in attributes['labels']:
            if interesting_concept == label:
                for type_ in attributes['types']:
                    types.append(type_)
    for type_ in types:
        typesdict[type_] = types.count(type_)
    interesting_type = max(typesdict, key=lambda i: [1])
    return interesting_concept, interesting_type

In [61]:
list_of_dict = create_concept_dictionaries(list_of_json_files)
concept_index = concept_indexer(list_of_dict)
term_idf_index = term_idf_indexer(list_of_idf_files)
input_index = input_indexer(text)
matched_terms_dic = match_terms(input_index, term_idf_index)
concept, category_type = define_respons_concept(matched_terms_dic)

In [62]:
print(concept, category_type)

american PopulatedPlace
